In [2]:
!pip install transformers reportlab qrcode pillow

from transformers import pipeline
import os
import random  # Import the random module

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as ReportLabImage, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.graphics.shapes import Drawing, Wedge, Rect, String  # CHANGED
from reportlab.graphics import renderPDF
import qrcode
from io import BytesIO
from PIL import Image
from google.colab import files

# --- 1. Setting up the Model (Hugging Face Pipeline) ---
model_name = "google/flan-t5-small"  # A smaller, faster T5 model
try:
    text_improver = pipeline('text2text-generation', model=model_name)
    print(f"Model '{model_name}' loaded successfully.")
except Exception as e:
    print(f"Error loading model '{model_name}': {e}")
    print("Please make sure you have transformers installed and the model name is correct.")
    print("Try running: pip install transformers")
    exit()

# --- 2. Industry-Specific Templates ---

RESUME_TEMPLATES = {
    "Software Engineering": {
        "SectionOrder": ["Summary", "Skills", "Experience", "Education", "Projects"],
        "Keywords": "Python, Java, Cloud Computing, Machine Learning, Agile, DevOps",
        "BulletPointStyle": "Focus on technical achievements and quantifiable results.",
        "ThemeColors": [colors.blue, colors.darkblue, colors.royalblue]  # Add theme colors
    },
    "Marketing": {
        "SectionOrder": ["Summary", "Experience", "Skills", "Education"],
        "Keywords": "Digital Marketing, SEO, Social Media, Content Marketing, Analytics, Branding",
        "BulletPointStyle": "Highlight marketing campaigns, ROI, and audience engagement.",
        "ThemeColors": [colors.orange, colors.darkorange, colors.goldenrod]  # Add theme colors
    },
    "Business": {
        "SectionOrder": ["Summary", "Experience", "Education", "Skills"],
        "Keywords": "Leadership, Management, Finance, Strategy, Consulting, Analysis",
        "BulletPointStyle": "Emphasize leadership roles, strategic initiatives, and business outcomes.",
        "ThemeColors": [colors.green, colors.darkgreen, colors.seagreen]  # Add theme colors
    },
    "Mechanical Engineering": {
        "SectionOrder": ["Summary", "Skills", "Experience", "Projects", "Education"],
        "Keywords": "CAD, FEA, Thermodynamics, Fluid Mechanics, Design, Manufacturing",
        "BulletPointStyle": "Emphasize design, manufacturing, and problem-solving skills.",
        "ThemeColors": [colors.gray, colors.dimgray, colors.lightslategray]  # Add theme colors
    }
}

# --- 3. Helper Lists and Improved User Input ---

STRONG_ACTION_VERBS = [
    "Achieved", "Administered", "Analyzed", "Authored", "Collaborated", "Communicated", "Coordinated", "Created",
    "Delegated", "Designed", "Developed", "Directed", "Enhanced", "Established", "Evaluated", "Executed",
    "Generated", "Implemented", "Improved", "Increased", "Initiated", "Innovated", "Managed", "Mentored",
    "Negotiated", "Operated", "Organized", "Oversaw", "Performed", "Planned", "Presented", "Reduced",
    "Researched", "Resolved", "Streamlined", "Supervised", "Trained", "Utilized"
]

def get_user_input():
    """Collects user input for resume sections with targeted prompts, validation, and default values."""

    user_data = {}

    print("\n--- Personal Information ---")
    user_data['name'] = input("Full Name (e.g., John Doe, default: John Doe): ") or "John Doe"
    user_data['email'] = input("Email (e.g., john.doe@example.com, default: john.doe@example.com): ") or "john.doe@example.com"
    user_data['phone'] = input("Phone Number (e.g., 123-456-7890, default: 123-456-7890): ") or "123-456-7890"
    user_data['linkedin'] = input("LinkedIn Profile URL (Optional, e.g., linkedin.com/in/johndoe, default: linkedin.com/in/johndoe): ") or "linkedin.com/in/johndoe"
    user_data['github'] = input("GitHub Profile URL (Optional, e.g., github.com/johndoe, default: github.com/johndoe): ") or "github.com/johndoe"
    user_data['location'] = input("Location (City, State, e.g., Anytown, CA, default: Anytown, CA): ") or "Anytown, CA"

    print("\n--- Job Preferences ---")
    user_data['company_type'] = input("Target Company Type (e.g., Software Engineering, Marketing, Business, Mechanical Engineering, default: Software Engineering): ") or "Software Engineering"
    if user_data['company_type'] not in RESUME_TEMPLATES:
        print(f"Warning: '{user_data['company_type']}' is not a recognized template. Using default 'Software Engineering'.")
        user_data['company_type'] = "Software Engineering"
    user_data['keywords'] = input(f"Keywords Relevant to Your Target Industry (e.g., {RESUME_TEMPLATES[user_data['company_type']]['Keywords']}, default: ): ") or RESUME_TEMPLATES[user_data['company_type']]['Keywords']

    print("\n--- Summary/Objective ---")
    #Increased word count for improved results
    user_data['summary'] = input("Summary/Objective (A compelling overview highlighting key skills and career goals, aim for multiple sentences to be more than 3 lines in the resume, default: Highly motivated professional seeking a challenging role...): ") or "Highly motivated professional seeking a challenging role where I can leverage my technical skills and experience. Seeking to contribute to a dynamic team environment. Possessing strong analytical and problem-solving abilities. Passionate about innovation and continuous learning. Committed to delivering high-quality results and exceeding expectations. Experience in working with diverse teams to achieve common goals."

    print("\n--- Work Experience (Enter each job separately) ---")
    user_data['experience'] = []
    while True:
        print("\n--- New Job ---")
        job = {}
        job['title'] = input("Job Title (e.g., Software Engineer, default: Software Engineer): ") or "Software Engineer"
        job['company'] = input("Company (e.g., Google, default: Google): ") or "Google"
        job['dates'] = input("Dates of Employment (e.g., Jan 2022 - Present or Jan 2022 - Dec 2023, default: Jan 2022 - Present): ") or "Jan 2022 - Present"
        job['description'] = input(f"Job Description (Use strong action verbs, quantify results, aim for 4-6 bullet points separated by newline, e.g., Developed new features resulting in X% increase in Y\\nImproved system performance by Z%, default: ): ") or "Developed new features resulting in 20% increase in user engagement\nImproved system performance by 15%\nCollaborated with team on project X\nImplemented agile methodologies"
        job['description_list'] = [item.strip() for item in job['description'].split('\n')]
        user_data['experience'].append(job)

        another_job = input("Add another job? (yes/no, default: no): ").lower() or "no"
        if another_job != 'yes':
            break

    print("\n--- Education (Enter each degree/certificate separately) ---")
    user_data['education'] = []
    while True:
        print("\n--- New Education Item ---")
        education = {}
        #Word counts increased for better results
        education['degree'] = input("Degree/Certificate (e.g., Bachelor of Science in Computer Science, default: Bachelor of Science in Computer Science): ") or "Bachelor of Science in Computer Science"
        education['institution'] = input("Institution (e.g., Stanford University, default: Stanford University): ") or "Stanford University"
        education['dates'] = input("Dates Attended (e.g., Sep 2018 - May 2022, default: Sep 2018 - May 2022): ") or "Sep 2018 - May 2022"
        #Increased word count for better results
        education['description'] = input("Description (Optional - GPA, relevant coursework, honors, e.g., GPA: 3.8, Relevant Coursework: Data Structures, Algorithms, Honors: Dean's List, default: ): ") or "GPA: 3.8, Relevant Coursework: Data Structures, Algorithms, Honors: Dean's List, President's Scholar"
        user_data['education'].append(education)

        another_edu = input("Add another education item? (yes/no, default: no): ").lower() or "no"
        if another_edu != 'yes':
            break

    print("\n--- Skills ---")
    user_data['skills_string'] = input("List your skills (separated by commas, e.g., Python, Java, SQL, default: ): ") or RESUME_TEMPLATES[user_data['company_type']]['Keywords']
    user_data['skills'] = [skill.strip() for skill in user_data['skills_string'].split(',')]

    # Skill Ratings with Input Validation
    user_data['skill_ratings'] = {}
    print("\nRate your proficiency in each skill (1-5, 5 being expert):")
    for skill in user_data['skills']:
        while True:
            try:
                rating = int(input(f"  {skill} (default: 4): ") or 4)  # Default rating of 4
                if 1 <= rating <= 5:
                    user_data['skill_ratings'][skill] = rating
                    break
                else:
                    print("Rating must be between 1 and 5.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    #Project Input
    print("\n--- Projects (Optional) ---")
    user_data['projects'] = input("List your projects (separated by commas, or leave blank, default: ): ") or ""

    # Add font selection option
    print("\n--- Font Options ---")
    user_data['font'] = input("Choose a font (Helvetica, Times-Roman, Courier, default: Helvetica): ") or 'Helvetica'
    while user_data['font'] not in ['Helvetica', 'Times-Roman', 'Courier']:
        print("Invalid font choice. Please select from Helvetica, Times-Roman, or Courier.")
        user_data['font'] = input("Choose a font (Helvetica, Times-Roman, Courier, default: Helvetica): ") or 'Helvetica'

    return user_data

# --- 3. Refined Text Improvement Function ---

def improve_text(text, company_type, keywords):
    """Improves the given text based on the company type and keywords using the text2text-generation model."""
    prompt = f"""Rewrite the following text to be more impactful and tailored for a resume targeting a {company_type} company.
    Incorporate the following keywords: {keywords}.
    Focus on quantifiable achievements and use strong action verbs. Keep it concise and professional: {text}"""
    try:
        improved_text = text_improver(prompt, max_length=300, num_return_sequences=1)[0]['generated_text'] #Increased max length and prompt to allow greater output in text
        return improved_text
    except Exception as e:
        print(f"Error improving text: {e}")
        return text  # Return original text if improvement fails

# --- 4. PDF Generation with Enhanced Styling and Randomized Style ---

def create_skill_bar_chart(skill, rating, theme_color, width=3*inch, height=0.2*inch): #Slimmer bars
    """Creates a horizontal bar chart for a single skill with label and box."""
    drawing = Drawing(width, height)
    max_rating = 5
    bar_width = width / max_rating
    x = 0
    bar = Rect(x, 0, bar_width * rating, height, fillColor=theme_color)
    drawing.add(bar)

    # Add rating label inside the bar
    label_text = f"{rating}/5"
    label_color = colors.white  # Choose a color that contrasts with the bar

    text_x = x + bar_width * rating / 2  # Center of the bar
    text_y = height / 2  # Center vertically

    label = String(text_x, text_y, label_text, textAnchor='middle', fillColor=label_color)
    drawing.add(label)

    # Add a box around the bar chart
    box = Rect(0, 0, width, height, strokeColor=colors.black, fillColor=None)
    box.strokeWidth = 0.5  #Make the boxes slimmer as well
    drawing.add(box)

    return drawing

def create_qr_code(url, size=1*inch):
    """Creates a QR code."""
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=8,
        border=4,
    )
    qr.add_data(url)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")
    img_buffer = BytesIO()
    img.save(img_buffer, format="PNG")
    img_buffer.seek(0)
    return img_buffer


def generate_pdf_resume(user_data, filename="resume.pdf"):
    """Generates a PDF resume with enhanced styling and randomized style elements."""
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()

    # --- Get template based on company type ---
    template = RESUME_TEMPLATES.get(user_data['company_type'], RESUME_TEMPLATES['Software Engineering']) # Default if not found
    theme_colors = template["ThemeColors"]
    selected_theme_color = random.choice(theme_colors)

    # --- Randomized Style Elements ---
    #font_options = ['Helvetica', 'Times-Roman', 'Courier']
    #selected_font = random.choice(font_options)
    selected_font = user_data['font'] # Use user-selected font
    bullet_colors = [colors.black, colors.gray, colors.dimgray]
    selected_bullet_color = random.choice(bullet_colors)


    # --- Custom Styles ---
    if 'Bullet' not in styles.byName:  # Check if 'Bullet' style exists
        styles.add(ParagraphStyle(name='Bullet', parent=styles['Normal'], leftIndent=0.2*inch, bulletIndent=0.1*inch, spaceBefore=6, spaceAfter=6, textColor=selected_bullet_color, fontName=selected_font))
    if 'SectionTitle' not in styles.byName:
        styles.add(ParagraphStyle(name='SectionTitle', parent=styles['Heading2'], textColor=selected_theme_color, fontName=selected_font))
    if 'JobTitle' not in styles.byName:
        styles.add(ParagraphStyle(name='JobTitle', parent=styles['Normal'], fontName=selected_font, spaceAfter=3))
    if 'CompanyDates' not in styles.byName:
        styles.add(ParagraphStyle(name='CompanyDates', parent=styles['Normal'], fontSize=10, textColor=colors.gray, fontName=selected_font)) #Subtle company/dates
    if 'SkillName' not in styles.byName:
        styles.add(ParagraphStyle(name='SkillName', parent=styles['Normal'], fontName=selected_font, fontSize=10))


    story = []

    def add_border(canvas, doc):
        canvas.saveState()
        canvas.setStrokeColorRGB(0, 0, 0)
        canvas.setLineWidth(3)  #Thinner border
        canvas.rect(inch, inch, doc.width, doc.height, stroke=1, fill=0)
        canvas.restoreState()


    # --- Header Section ---
    story.append(Paragraph(user_data['name'], styles['Heading1']))
    contact_info = f"Email: {user_data['email']} | Phone: {user_data['phone']}"
    if user_data['linkedin']:
        contact_info += f" | LinkedIn: <a href='{user_data['linkedin']}'>{user_data['linkedin']}</a>"
    if user_data['github']:
        contact_info += f" | GitHub: <a href='{user_data['github']}'>{user_data['github']}</a>"
    contact_info += f" | Location: {user_data['location']}"
    story.append(Paragraph(contact_info, styles['Normal']))
    story.append(Spacer(1, 0.1*inch))

    # QR code
    if user_data['linkedin']:
        qr_code_buffer = create_qr_code(user_data['linkedin'])
        story.append(ReportLabImage(qr_code_buffer, width=1*inch, height=1*inch))
        story.append(Spacer(1, 0.1*inch))

    # --- Section Order based on Template ---
    section_order = template["SectionOrder"]

    # --- Prepare Skills Data for later insertion ---
    skills_data = None
    if "Skills" in section_order:
        section_order.remove("Skills")
        skill_ratings = user_data['skill_ratings']
        skills_data = []  # Store each skill and its chart in a list
        for skill in user_data['skills']:
            rating = skill_ratings.get(skill, 4)  # Default to 4 if rating is missing
            skill_chart = create_skill_bar_chart(skill, rating, selected_theme_color)
            skills_data.append((skill, skill_chart))


    for section in section_order:
        if section == "Summary":
            improved_summary = improve_text(user_data['summary'], user_data['company_type'], user_data['keywords'])
            story.append(Paragraph("<b>Summary</b>", styles['SectionTitle']))
            story.append(Paragraph(improved_summary, styles['Normal']))
            story.append(Spacer(1, 0.2*inch))
        elif section == "Experience":
            story.append(Paragraph("<b>Experience</b>", styles['SectionTitle']))
            for job in user_data['experience']:
                story.append(Paragraph(job['title'], styles['JobTitle']))
                story.append(Paragraph(f"{job['company']} ({job['dates']})", styles['CompanyDates'])) #Styling!
                for item in job['description_list']:
                    improved_item = improve_text(item, user_data['company_type'], user_data['keywords']) #Improve bullet point
                    story.append(Paragraph(f"• {improved_item}", styles['Bullet']))
                story.append(Spacer(1, 0.1*inch))
            story.append(Spacer(1, 0.2*inch))
        elif section == "Education":
            story.append(Paragraph("<b>Education</b>", styles['SectionTitle']))
            for education in user_data['education']:
                story.append(Paragraph(education['degree'], styles['JobTitle'])) #Reusing JobTitle style
                story.append(Paragraph(f"{education['institution']} ({education['dates']})", styles['CompanyDates']))
                story.append(Paragraph(education['description'], styles['Normal']))
                story.append(Spacer(1, 0.1*inch))
            story.append(Spacer(1, 0.2*inch))
        elif section == "Projects":
            #This part is intentionally left blank since it is an optional feature and could be left like that
            #If projects section is ever implemented you can include the data in this section in an if else format
            pass


    # --- Add Skills Section to the Bottom ---
    if skills_data:
        story.append(Paragraph("<b>Skills</b>", styles['SectionTitle']))
        for skill, skill_chart in skills_data:
            story.append(Paragraph(skill, styles['SkillName']))  # Add skill name
            story.append(skill_chart)  # Add the bar chart for the skill
            story.append(Spacer(1, 0.05*inch))  # Small space between skills

    doc.build(story, onFirstPage=add_border, onLaterPages=add_border)
    return filename

# --- 5. Main Execution ---

if __name__ == "__main__":
    print("--- Resume Generator ---")

    user_data = get_user_input()

    pdf_filename = generate_pdf_resume(user_data)

    print(f"Resume generated successfully as {pdf_filename}!")
    files.download(pdf_filename)

    print("--- Done! ---")

Device set to use cpu


Model 'google/flan-t5-small' loaded successfully.
--- Resume Generator ---

--- Personal Information ---
Full Name (e.g., John Doe, default: John Doe): 
Email (e.g., john.doe@example.com, default: john.doe@example.com): 
Phone Number (e.g., 123-456-7890, default: 123-456-7890): 
LinkedIn Profile URL (Optional, e.g., linkedin.com/in/johndoe, default: linkedin.com/in/johndoe): 
GitHub Profile URL (Optional, e.g., github.com/johndoe, default: github.com/johndoe): 
Location (City, State, e.g., Anytown, CA, default: Anytown, CA): 

--- Job Preferences ---
Target Company Type (e.g., Software Engineering, Marketing, Business, Mechanical Engineering, default: Software Engineering): 
Keywords Relevant to Your Target Industry (e.g., Python, Java, Cloud Computing, Machine Learning, Agile, DevOps, default: ): 

--- Summary/Objective ---
Summary/Objective (A compelling overview highlighting key skills and career goals, aim for multiple sentences to be more than 3 lines in the resume, default: Highl

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--- Done! ---
